# 🔍 03 - Análise de Padrões com Regex

## 📖 Visão Geral

Este notebook realiza análise baseada em expressões regulares para identificar nanomateriais, propriedades funcionais e técnicas de preparação nos textos científicos.

### 🎯 Responsabilidades

- ✅ Identificação de nanomateriais usando padrões regex avançados
- ✅ Extração de propriedades funcionais de tintas e revestimentos
- ✅ Detecção de técnicas de preparação e aplicação
- ✅ Análise de co-ocorrências entre materiais e propriedades
- ✅ Geração de matrizes de relacionamento
- ✅ Preparação de dados para análise Gemini

### 📦 Dependências

- pandas
- numpy
- re (regex)
- json
- plotly (para visualizações)
- networkx (para análise de redes)

### 🔗 Notebooks Relacionados

- **Anterior**: `02_carregamento_dados.ipynb`
- **Próximo**: `04_analise_gemini.ipynb`

In [13]:
# 📦 Imports e carregamento de dados

import pandas as pd
import numpy as np
import re
import json
import os
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Set, Optional, Any
from collections import Counter, defaultdict
import itertools
import warnings
warnings.filterwarnings('ignore')

# Tentar importar bibliotecas opcionais
try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    PLOTLY_AVAILABLE = True
    print("✅ Plotly disponível para visualizações")
except ImportError:
    PLOTLY_AVAILABLE = False
    print("⚠️ Plotly não disponível - visualizações desabilitadas")

try:
    import networkx as nx
    NETWORKX_AVAILABLE = True
    print("✅ NetworkX disponível para análise de redes")
except ImportError:
    NETWORKX_AVAILABLE = False
    print("⚠️ NetworkX não disponível - análise de redes desabilitada")

print("\n📦 Imports concluídos")

✅ Plotly disponível para visualizações
✅ NetworkX disponível para análise de redes

📦 Imports concluídos


## 🔧 Carregamento de Configurações e Dados

In [14]:
# 🔧 Carregamento de configurações e dados

# Carregar configurações do sistema
config_file = 'config_sistema.json'
if os.path.exists(config_file):
    with open(config_file, 'r', encoding='utf-8') as f:
        config_data = json.load(f)
    
    SISTEMA_CONFIGURADO = config_data.get('sistema_configurado', False)
    PATHS = config_data.get('paths', {})
    CONFIG = config_data.get('config', {})
    NANOMATERIAIS_DB = config_data.get('nanomateriais', {})
    PALAVRAS_CHAVE_DB = config_data.get('palavras_chave', {})
    
    print("✅ Configurações carregadas do sistema")
    print(f"   Sistema configurado: {SISTEMA_CONFIGURADO}")
    print(f"   Nanomateriais disponíveis: {len(NANOMATERIAIS_DB)} categorias")
    print(f"   Palavras-chave disponíveis: {len(PALAVRAS_CHAVE_DB)} categorias")
else:
    print("❌ Arquivo de configuração não encontrado!")
    print("💡 Execute primeiro o notebook '01_configuracao_sistema.ipynb'")
    raise FileNotFoundError("Configuração do sistema necessária")

# Tentar carregar dados do notebook anterior
processed_dir = PATHS.get('processed', './processed/')
latest_data_file = os.path.join(processed_dir, 'dataset_wos_latest.csv')

if os.path.exists(latest_data_file):
    print(f"\n📊 Carregando dados processados...")
    df_data = pd.read_csv(latest_data_file, encoding='utf-8-sig')
    print(f"✅ Dados carregados: {len(df_data):,} registros × {len(df_data.columns)} colunas")
    
    # Verificar colunas essenciais
    colunas_essenciais = ['Article Title', 'Abstract']
    colunas_presentes = [col for col in colunas_essenciais if col in df_data.columns]
    print(f"📋 Colunas essenciais presentes: {len(colunas_presentes)}/{len(colunas_essenciais)}")
    
    if len(colunas_presentes) < len(colunas_essenciais):
        print(f"⚠️ Colunas faltantes: {set(colunas_essenciais) - set(colunas_presentes)}")
    
    # Verificar colunas de Open Access disponíveis
    open_access_columns = [col for col in df_data.columns if 'open' in col.lower() or 'access' in col.lower()]
    print(f"\n🔍 Colunas relacionadas ao Open Access encontradas: {open_access_columns}")
    
    # Verificar se há coluna de filtro
    if 'filtrado_open_access' in df_data.columns:
        oa_filtered = df_data['filtrado_open_access'].sum()
        print(f"📊 Registros já filtrados por Open Access: {oa_filtered:,}")
else:
    print("❌ Dados processados não encontrados!")
    print("💡 Execute primeiro o notebook '02_carregamento_dados.ipynb'")
    raise FileNotFoundError("Dados processados necessários")

✅ Configurações carregadas do sistema
   Sistema configurado: True
   Nanomateriais disponíveis: 6 categorias
   Palavras-chave disponíveis: 4 categorias

📊 Carregando dados processados...


✅ Dados carregados: 9,046 registros × 76 colunas
📋 Colunas essenciais presentes: 2/2

🔍 Colunas relacionadas ao Open Access encontradas: ['Early Access Date', 'Open Access Designations', 'filtrado_open_access']
📊 Registros já filtrados por Open Access: 9,046


In [15]:
# 🔎 Verificar e usar dados filtrados por Open Access se disponíveis
if 'Open Access' in df_data.columns:
    # Aplicar filtro de Open Access
    df_clean = df_data[df_data['Open Access'] == True]
    print(f"🔒 Filtro de Open Access aplicado: {len(df_clean):,} registros de Open Access encontrados")
    print(f"📉 Registros removidos (não Open Access): {len(df_data) - len(df_clean):,}")
else:
    # Verificar outras colunas possíveis de Open Access
    possiveis_colunas_oa = ['Open Access Designations', 'OA', 'Acesso Aberto']
    coluna_oa_encontrada = None
    
    for col in possiveis_colunas_oa:
        if col in df_data.columns:
            coluna_oa_encontrada = col
            break
    
    if coluna_oa_encontrada:
        # Filtrar por registros com classificação de Open Access não vazia
        df_clean = df_data[df_data[coluna_oa_encontrada].notna()]
        df_clean = df_clean[df_clean[coluna_oa_encontrada].astype(str).str.strip() != '']
        df_clean = df_clean[~df_clean[coluna_oa_encontrada].astype(str).str.lower().isin(['nan', 'none', 'unknown'])]
        print(f"🔒 Filtro de Open Access aplicado via coluna '{coluna_oa_encontrada}': {len(df_clean):,} registros")
        print(f"📉 Registros removidos: {len(df_data) - len(df_clean):,}")
    else:
        # Verificar se os dados já vieram filtrados (coluna filtrado_open_access)
        if 'filtrado_open_access' in df_data.columns:
            df_clean = df_data.copy()
            registros_oa = df_data['filtrado_open_access'].sum()
            print(f"🔍 Dados já filtrados por Open Access: {len(df_clean):,} registros ({registros_oa:,} marcados como Open Access)")
        else:
            df_clean = df_data.copy()
            print("⚠️ Nenhuma coluna de Open Access encontrada. Usando todos os dados.")

print(f"📊 Dataset final para análise regex: {len(df_clean):,} registros")

# Verificar se realmente temos dados de Open Access
if 'filtrado_open_access' in df_clean.columns:
    oa_marcados = df_clean['filtrado_open_access'].sum()
    if oa_marcados == len(df_clean):
        print(f"✅ Confirmação: Todos os {len(df_clean):,} registros são Open Access")
    else:
        print(f"⚠️ Aviso: {oa_marcados:,} de {len(df_clean):,} registros marcados como Open Access")
        
# 🔎 Aplicar filtro de Open Access se disponível
print("\n🔒 VERIFICANDO E APLICANDO FILTRO DE OPEN ACCESS")
print("=" * 50)

# Verificar se dados já foram filtrados
if 'filtrado_open_access' in df_data.columns:
    # Dados já foram filtrados no notebook anterior
    open_access_count = df_data['filtrado_open_access'].sum()
    if open_access_count == len(df_data):
        print(f"✅ Dados já estão filtrados por Open Access: {len(df_data):,} registros (100%)")
        df_clean = df_data.copy()
    else:
        print(f"⚠️ Dados parcialmente filtrados. Aplicando filtro completo...")
        df_clean = df_data[df_data['filtrado_open_access'] == True]
        print(f"📊 Após filtro: {len(df_clean):,} registros Open Access")
else:
    # Tentar encontrar colunas de Open Access
    open_access_columns = [col for col in df_data.columns if 'open' in col.lower() and 'access' in col.lower()]
    
    if open_access_columns:
        oa_col = open_access_columns[0]
        print(f"🔍 Usando coluna: '{oa_col}'")
        
        # Filtrar por Open Access
        initial_count = len(df_data)
        df_clean = df_data[df_data[oa_col].notna()]
        df_clean = df_clean[df_clean[oa_col].astype(str).str.strip() != '']
        df_clean = df_clean[~df_clean[oa_col].astype(str).str.lower().isin(['nan', 'none', 'unknown', ''])]
        
        final_count = len(df_clean)
        removed_count = initial_count - final_count
        
        print(f"📊 Registros iniciais: {initial_count:,}")
        print(f"📊 Registros finais: {final_count:,}")
        print(f"🗑️ Registros removidos: {removed_count:,} ({removed_count/initial_count*100:.1f}%)")
        
        # Adicionar flag
        df_clean['filtrado_open_access'] = True
    else:
        print("⚠️ Nenhuma coluna de Open Access encontrada. Usando todos os dados.")
        df_clean = df_data.copy()
        df_clean['filtrado_open_access'] = False

# Atualizar df_data para usar dados filtrados no resto do notebook
df_data = df_clean
print(f"\n✅ Dataset final para análise: {len(df_data):,} registros")
print(f"📊 Colunas disponíveis: {len(df_data.columns)}")

🔒 Filtro de Open Access aplicado via coluna 'Open Access Designations': 9,046 registros
📉 Registros removidos: 0
📊 Dataset final para análise regex: 9,046 registros
✅ Confirmação: Todos os 9,046 registros são Open Access

🔒 VERIFICANDO E APLICANDO FILTRO DE OPEN ACCESS
✅ Dados já estão filtrados por Open Access: 9,046 registros (100%)

✅ Dataset final para análise: 9,046 registros
📊 Colunas disponíveis: 76


In [16]:
# 📊 Resumo do carregamento e filtro
print("\n📋 RESUMO DO CARREGAMENTO DE DADOS:")
print("=" * 40)
print(f"📁 Dados carregados de: {latest_data_file}")
print(f"📊 Total de registros: {len(df_data):,}")
print(f"📋 Total de colunas: {len(df_data.columns)}")

if 'filtrado_open_access' in df_data.columns:
    open_access_aplicado = df_data['filtrado_open_access'].any()
    if open_access_aplicado:
        print(f"✅ Filtro Open Access: APLICADO")
        oa_count = df_data['filtrado_open_access'].sum()
        print(f"🔒 Registros Open Access: {oa_count:,} ({oa_count/len(df_data)*100:.1f}% do total)")
    else:
        print(f"⚠️ Filtro Open Access: NÃO APLICADO")
else:
    print(f"⚠️ Filtro Open Access: COLUNA NÃO ENCONTRADA")

print(f"\n🏁 Dataset pronto para análise regex com {len(df_data):,} registros")


📋 RESUMO DO CARREGAMENTO DE DADOS:
📁 Dados carregados de: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_wos_latest.csv
📊 Total de registros: 9,046
📋 Total de colunas: 76
✅ Filtro Open Access: APLICADO
🔒 Registros Open Access: 9,046 (100.0% do total)

🏁 Dataset pronto para análise regex com 9,046 registros


In [17]:
# 🎯 Definição de padrões regex avançados

class RegexAnalyzer:
    """Classe para análise de padrões regex em textos científicos"""
    
    def __init__(self, nanomateriais_db: Dict, palavras_chave_db: Dict):
        self.nanomateriais_db = nanomateriais_db
        self.palavras_chave_db = palavras_chave_db
        self.compiled_patterns = {}
        self._compile_patterns()
    
    def _compile_patterns(self):
        """Compila todos os padrões regex para melhor performance"""
        
        print("🔧 Compilando padrões regex...")
        
        # Padrões para nanomateriais
        for categoria, materiais in self.nanomateriais_db.items():
            patterns = []
            for material in materiais:
                # Padrão básico
                pattern = rf'\b{re.escape(material)}\b'
                patterns.append(pattern)
                
                # Variações com nano-prefix
                if not material.lower().startswith('nano'):
                    nano_pattern = rf'\bnano[-\s]?{re.escape(material)}\b'
                    patterns.append(nano_pattern)
                
                # Padrões para abreviações químicas
                if len(material) <= 6 and material.isupper():
                    abbrev_pattern = rf'\b{re.escape(material)}(?:[\s-]?NPs?|[\s-]?nanoparticles?)?\b'
                    patterns.append(abbrev_pattern)
            
            # Compilar padrão combinado
            combined_pattern = '|'.join(patterns)
            self.compiled_patterns[f'nano_{categoria}'] = re.compile(
                combined_pattern, re.IGNORECASE | re.MULTILINE
            )
        
        # Padrões para palavras-chave funcionais
        for categoria, palavras in self.palavras_chave_db.items():
            patterns = []
            for palavra in palavras:
                # Padrão básico com word boundaries
                pattern = rf'\b{re.escape(palavra)}\b'
                patterns.append(pattern)
                
                # Variações plurais e compostas
                if not palavra.endswith('s'):
                    plural_pattern = rf'\b{re.escape(palavra)}s?\b'
                    patterns.append(plural_pattern)
            
            combined_pattern = '|'.join(patterns)
            self.compiled_patterns[f'func_{categoria}'] = re.compile(
                combined_pattern, re.IGNORECASE | re.MULTILINE
            )
        
        # Padrões especiais para medidas e valores
        self.compiled_patterns['medidas'] = re.compile(
            r'\b(?:\d+[.,]?\d*)\s*(?:nm|μm|mm|cm|m|ml|l|g|kg|%|°C|K|Pa|MPa|GPa)\b',
            re.IGNORECASE
        )
        
        # Padrões para métodos de síntese
        synthesis_methods = [
            'sol-gel', 'chemical vapor deposition', 'CVD', 'physical vapor deposition', 'PVD',
            'electrochemical', 'hydrothermal', 'solvothermal', 'precipitation',
            'co-precipitation', 'ball milling', 'spray pyrolysis'
        ]
        synthesis_pattern = '|'.join([rf'\b{re.escape(method)}\b' for method in synthesis_methods])
        self.compiled_patterns['sintese'] = re.compile(synthesis_pattern, re.IGNORECASE)
        
        print(f"✅ {len(self.compiled_patterns)} padrões regex compilados")
    
    def extract_matches(self, text: str, pattern_category: str) -> List[str]:
        """Extrai todas as correspondências de um padrão específico"""
        if not text or not isinstance(text, str):
            return []
        
        if pattern_category not in self.compiled_patterns:
            return []
        
        matches = self.compiled_patterns[pattern_category].findall(text)
        # Limpar e normalizar matches
        clean_matches = [match.strip().lower() for match in matches if match.strip()]
        return list(set(clean_matches))  # Remover duplicatas
    
    def extract_all_patterns(self, text: str) -> Dict[str, List[str]]:
        """Extrai todos os padrões de um texto"""
        results = {}
        for pattern_name in self.compiled_patterns.keys():
            results[pattern_name] = self.extract_matches(text, pattern_name)
        return results
    
    def analyze_text_comprehensive(self, text: str) -> Dict[str, any]:
        """Análise abrangente de um texto"""
        if not text or not isinstance(text, str):
            return {}
        # Iniciando análise abrangente
        analysis = {
            'text_length': len(text),
            'word_count': len(text.split()),
            'patterns_found': {},
            'total_matches': 0,
            'nano_relevance_score': 0,
            'functional_relevance_score': 0
        }
        
        # Extrair todos os padrões
        all_patterns = self.extract_all_patterns(text)
        analysis['patterns_found'] = all_patterns
        
        # Calcular scores de relevância
        nano_matches = 0
        func_matches = 0
        
        for pattern_name, matches in all_patterns.items():
            match_count = len(matches)
            analysis['total_matches'] += match_count
            
            if pattern_name.startswith('nano_'):
                nano_matches += match_count
            elif pattern_name.startswith('func_'):
                func_matches += match_count
        
        # Normalizar scores (0-100)
        if analysis['word_count'] > 0:
            analysis['nano_relevance_score'] = min(100, (nano_matches / analysis['word_count']) * 1000)
            analysis['functional_relevance_score'] = min(100, (func_matches / analysis['word_count']) * 1000)
        
        return analysis

# Inicializar o analisador
print("🚀 Inicializando analisador regex...")
regex_analyzer = RegexAnalyzer(NANOMATERIAIS_DB, PALAVRAS_CHAVE_DB)
print("✅ Analisador regex pronto")

🚀 Inicializando analisador regex...
🔧 Compilando padrões regex...
✅ 12 padrões regex compilados
✅ Analisador regex pronto


## 🔄 Análise em Lote dos Dados

In [18]:
# 🔄 Análise em lote dos dados

def analyze_dataset_regex(df: pd.DataFrame, analyzer: RegexAnalyzer, 
                         batch_size: int = 100) -> pd.DataFrame:
    """Analisa todo o dataset usando regex em lotes para melhor performance"""
    
    print("🔄 INICIANDO ANÁLISE REGEX EM LOTE")
    print("=" * 40)
    
    total_records = len(df)
    print(f"📊 Total de registros: {total_records:,}")
    print(f"📦 Tamanho do lote: {batch_size}")
    
    # Preparar DataFrame resultado
    df_result = df.copy()
    
    # Colunas para armazenar resultados
    resultado_colunas = [
        'nano_relevance_score', 'functional_relevance_score', 'total_matches',
        'nanomateriais_encontrados', 'propriedades_funcionais', 'metodos_sintese',
        'medidas_encontradas', 'analise_completa'
    ]
    
    for col in resultado_colunas:
        df_result[col] = None
    
    # Processar em lotes
    total_batches = (total_records + batch_size - 1) // batch_size
    
    for batch_idx in range(total_batches):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, total_records)
        
        print(f"\n📦 Processando lote {batch_idx + 1}/{total_batches} ({start_idx+1}-{end_idx})")
        
        # Processar registros do lote
        for idx in range(start_idx, end_idx):
            # Combinar título e abstract para análise
            titulo = str(df_result.iloc[idx].get('Article Title', ''))
            abstract = str(df_result.iloc[idx].get('Abstract', ''))
            texto_completo = f"{titulo} {abstract}".strip()
            
            if not texto_completo or texto_completo == 'nan nan':
                continue
            
            # Analisar texto
            analise = analyzer.analyze_text_comprehensive(texto_completo)
            
            # Extrair informações específicas
            patterns_found = analise.get('patterns_found', {})
            
            # Coletar nanomateriais
            nanomateriais = []
            for pattern_name, matches in patterns_found.items():
                if pattern_name.startswith('nano_'):
                    nanomateriais.extend(matches)
            
            # Coletar propriedades funcionais
            propriedades = []
            for pattern_name, matches in patterns_found.items():
                if pattern_name.startswith('func_'):
                    propriedades.extend(matches)
            
            # Armazenar resultados
            df_result.iloc[idx, df_result.columns.get_loc('nano_relevance_score')] = analise.get('nano_relevance_score', 0)
            df_result.iloc[idx, df_result.columns.get_loc('functional_relevance_score')] = analise.get('functional_relevance_score', 0)
            df_result.iloc[idx, df_result.columns.get_loc('total_matches')] = analise.get('total_matches', 0)
            df_result.iloc[idx, df_result.columns.get_loc('nanomateriais_encontrados')] = json.dumps(list(set(nanomateriais)), ensure_ascii=False)
            df_result.iloc[idx, df_result.columns.get_loc('propriedades_funcionais')] = json.dumps(list(set(propriedades)), ensure_ascii=False)
            df_result.iloc[idx, df_result.columns.get_loc('metodos_sintese')] = json.dumps(patterns_found.get('sintese', []), ensure_ascii=False)
            df_result.iloc[idx, df_result.columns.get_loc('medidas_encontradas')] = json.dumps(patterns_found.get('medidas', []), ensure_ascii=False)
            df_result.iloc[idx, df_result.columns.get_loc('analise_completa')] = json.dumps(analise, ensure_ascii=False)
        
        # Progresso
        progress = ((batch_idx + 1) / total_batches) * 100
        print(f"✅ Lote concluído ({progress:.1f}% total)")
    
    print(f"\n🎉 ANÁLISE REGEX CONCLUÍDA")
    print(f"📊 Registros processados: {total_records:,}")
    
    return df_result

# Executar análise em lote usando dados filtrados por Open Access
print("🚀 Iniciando análise regex do dataset filtrado...")
# Usar df_clean que contém os dados filtrados por Open Access
df_analyzed = analyze_dataset_regex(df_clean, regex_analyzer, batch_size=50)
print("✅ Análise regex concluída")

🚀 Iniciando análise regex do dataset filtrado...
🔄 INICIANDO ANÁLISE REGEX EM LOTE
📊 Total de registros: 9,046
📦 Tamanho do lote: 50

📦 Processando lote 1/181 (1-50)
✅ Lote concluído (0.6% total)

📦 Processando lote 2/181 (51-100)
✅ Lote concluído (0.6% total)

📦 Processando lote 2/181 (51-100)


✅ Lote concluído (1.1% total)

📦 Processando lote 3/181 (101-150)
✅ Lote concluído (1.7% total)

📦 Processando lote 4/181 (151-200)
✅ Lote concluído (2.2% total)

📦 Processando lote 5/181 (201-250)
✅ Lote concluído (2.8% total)

📦 Processando lote 6/181 (251-300)
✅ Lote concluído (1.7% total)

📦 Processando lote 4/181 (151-200)
✅ Lote concluído (2.2% total)

📦 Processando lote 5/181 (201-250)
✅ Lote concluído (2.8% total)

📦 Processando lote 6/181 (251-300)
✅ Lote concluído (3.3% total)

📦 Processando lote 7/181 (301-350)
✅ Lote concluído (3.3% total)

📦 Processando lote 7/181 (301-350)
✅ Lote concluído (3.9% total)

📦 Processando lote 8/181 (351-400)
✅ Lote concluído (3.9% total)

📦 Processando lote 8/181 (351-400)
✅ Lote concluído (4.4% total)

📦 Processando lote 9/181 (401-450)
✅ Lote concluído (4.4% total)

📦 Processando lote 9/181 (401-450)
✅ Lote concluído (5.0% total)

📦 Processando lote 10/181 (451-500)
✅ Lote concluído (5.0% total)

📦 Processando lote 10/181 (451-500)
✅ Lote c

## 📈 Estatísticas e Análise dos Resultados

In [19]:
# 📈 Estatísticas e análise dos resultados

def generate_regex_statistics(df: pd.DataFrame) -> Dict[str, any]:
    """Gera estatísticas abrangentes da análise regex"""
    
    print("📈 GERANDO ESTATÍSTICAS DA ANÁLISE REGEX")
    print("=" * 45)
    
    stats = {
        'total_registros': len(df),
        'registros_com_nano': 0,
        'registros_com_func': 0,
        'scores': {},
        'nanomateriais_frequencia': Counter(),
        'propriedades_frequencia': Counter(),
        'metodos_sintese_frequencia': Counter(),
        'coocorrencias': defaultdict(int)
    }
    
    # Análise de scores
    if 'nano_relevance_score' in df.columns:
        nano_scores = pd.to_numeric(df['nano_relevance_score'], errors='coerce').fillna(0)
        stats['registros_com_nano'] = (nano_scores > 0).sum()
        stats['scores']['nano'] = {
            'mean': float(nano_scores.mean()),
            'median': float(nano_scores.median()),
            'max': float(nano_scores.max()),
            'std': float(nano_scores.std())
        }
    
    if 'functional_relevance_score' in df.columns:
        func_scores = pd.to_numeric(df['functional_relevance_score'], errors='coerce').fillna(0)
        stats['registros_com_func'] = (func_scores > 0).sum()
        stats['scores']['functional'] = {
            'mean': float(func_scores.mean()),
            'median': float(func_scores.median()),
            'max': float(func_scores.max()),
            'std': float(func_scores.std())
        }
    
    print(f"📊 Registros com nanomateriais: {stats['registros_com_nano']:,} ({stats['registros_com_nano']/len(df)*100:.1f}%)")
    print(f"📊 Registros com propriedades funcionais: {stats['registros_com_func']:,} ({stats['registros_com_func']/len(df)*100:.1f}%)")
    
    # Análise de frequências
    print(f"\\n🔍 Analisando frequências...")
    
    for idx, row in df.iterrows():
        # Nanomateriais
        if pd.notna(row.get('nanomateriais_encontrados')):
            try:
                nanomateriais = json.loads(row['nanomateriais_encontrados'])
                for nano in nanomateriais:
                    stats['nanomateriais_frequencia'][nano] += 1
            except:
                pass
        
        # Propriedades funcionais
        if pd.notna(row.get('propriedades_funcionais')):
            try:
                propriedades = json.loads(row['propriedades_funcionais'])
                for prop in propriedades:
                    stats['propriedades_frequencia'][prop] += 1
            except:
                pass
        
        # Métodos de síntese
        if pd.notna(row.get('metodos_sintese')):
            try:
                metodos = json.loads(row['metodos_sintese'])
                for metodo in metodos:
                    stats['metodos_sintese_frequencia'][metodo] += 1
            except:
                pass
    
    # Top 10 de cada categoria
    print(f"\n🏆 TOP 10 NANOMATERIAIS:")
    for material, freq in stats['nanomateriais_frequencia'].most_common(10):
        print(f"   {material}: {freq:,} menções")
    
    print(f"\n🏆 TOP 10 PROPRIEDADES FUNCIONAIS:")
    for prop, freq in stats['propriedades_frequencia'].most_common(10):
        print(f"   {prop}: {freq:,} menções")
    
    print(f"\n🏆 TOP 10 MÉTODOS DE SÍNTESE:")
    for metodo, freq in stats['metodos_sintese_frequencia'].most_common(10):
        print(f"   {metodo}: {freq:,} menções")
    
    return stats

# Gerar estatísticas
regex_stats = generate_regex_statistics(df_analyzed)
print("\n✅ Estatísticas geradas com sucesso")

📈 GERANDO ESTATÍSTICAS DA ANÁLISE REGEX
📊 Registros com nanomateriais: 4,235 (46.8%)
📊 Registros com propriedades funcionais: 8,071 (89.2%)
\n🔍 Analisando frequências...

🏆 TOP 10 NANOMATERIAIS:
   tio2: 550 menções
   al2o3: 446 menções
   sio2: 383 menções
   silica: 382 menções
   graphene: 334 menções
   alumina: 314 menções
   nanocomposite: 296 menções
   zno: 264 menções
   silver: 259 menções
   zirconia: 256 menções

🏆 TOP 10 PROPRIEDADES FUNCIONAIS:
   coating: 6,062 menções
   coatings: 3,628 menções
   thin films: 602 menções
   sol-gel: 424 menções
   thermal barrier: 311 menções
   thin film: 296 menções
   biomedical: 274 menções
   spin coating: 244 menções
   superhydrophobic: 233 menções
   magnetic: 232 menções

🏆 TOP 10 MÉTODOS DE SÍNTESE:
   electrochemical: 862 menções
   sol-gel: 424 menções
   chemical vapor deposition: 164 menções
   hydrothermal: 134 menções
   precipitation: 127 menções
   pvd: 107 menções
   cvd: 105 menções
   physical vapor deposition: 80 

In [20]:
def analyze_cooccurrences(df: pd.DataFrame) -> Dict[str, any]:
    """Analisa co-ocorrências entre nanomateriais e propriedades funcionais"""
    
    print("🔗 ANALISANDO CO-OCORRÊNCIAS")
    print("=" * 30)
    
    cooccurrences = defaultdict(int)
    material_property_pairs = []
    
    for idx, row in df.iterrows():
        # Extrair nanomateriais e propriedades
        nanomateriais = []
        propriedades = []
        
        if pd.notna(row.get('nanomateriais_encontrados')):
            try:
                nanomateriais = json.loads(row['nanomateriais_encontrados'])
            except:
                pass
        
        if pd.notna(row.get('propriedades_funcionais')):
            try:
                propriedades = json.loads(row['propriedades_funcionais'])
            except:
                pass
        
        # Calcular co-ocorrências
        for nano in nanomateriais:
            for prop in propriedades:
                pair = f"{nano} + {prop}"
                cooccurrences[pair] += 1
                material_property_pairs.append((nano, prop))
    
    # Top 20 co-ocorrências
    print(f"\n🏆 TOP 20 CO-OCORRÊNCIAS:")
    for pair, count in Counter(cooccurrences).most_common(20):
        print(f"   {pair}: {count} registros")
    
    return {
        'cooccurrences': dict(cooccurrences),
        'total_pairs': len(material_property_pairs),
        'unique_pairs': len(set(material_property_pairs))
    }

def identify_high_relevance_records(df: pd.DataFrame, 
                                  nano_threshold: float = 10.0,
                                  func_threshold: float = 5.0) -> pd.DataFrame:
    """Identifica registros com alta relevância para nanotecnologia de tintas"""
    
    print(f"🎯 IDENTIFICANDO REGISTROS DE ALTA RELEVÂNCIA")
    print("=" * 50)
    
    # Filtros de relevância
    nano_scores = pd.to_numeric(df.get('nano_relevance_score', 0), errors='coerce').fillna(0)
    func_scores = pd.to_numeric(df.get('functional_relevance_score', 0), errors='coerce').fillna(0)
    
    # Diferentes níveis de relevância
    high_nano = df[nano_scores >= nano_threshold]
    high_func = df[func_scores >= func_threshold]
    high_both = df[(nano_scores >= nano_threshold) & (func_scores >= func_threshold)]
    
    print(f"📊 Registros com alta relevância nano (≥{nano_threshold}): {len(high_nano):,}")
    print(f"📊 Registros com alta relevância funcional (≥{func_threshold}): {len(high_func):,}")
    print(f"🎯 Registros com AMBAS as relevâncias altas: {len(high_both):,}")
    
    # Adicionar classificação de relevância
    df_result = df.copy()
    df_result['relevance_category'] = 'low'
    
    df_result.loc[nano_scores >= nano_threshold, 'relevance_category'] = 'high_nano'
    df_result.loc[func_scores >= func_threshold, 'relevance_category'] = 'high_functional'
    df_result.loc[(nano_scores >= nano_threshold) & (func_scores >= func_threshold), 'relevance_category'] = 'high_both'
    
    # Estatísticas por categoria
    category_counts = df_result['relevance_category'].value_counts()
    print("\n📈 DISTRIBUIÇÃO POR CATEGORIA:")
    for category, count in category_counts.items():
        percentage = (count / len(df_result)) * 100
        print(f"   {category}: {count:,} ({percentage:.1f}%)")
    
    return df_result

# Analisar co-ocorrências
cooccurrence_analysis = analyze_cooccurrences(df_analyzed)

# Identificar registros de alta relevância
df_with_relevance = identify_high_relevance_records(df_analyzed)
print("\n✅ Identificação de registros de alta relevância concluída")

🔗 ANALISANDO CO-OCORRÊNCIAS



🏆 TOP 20 CO-OCORRÊNCIAS:
   tio2 + coating: 387 registros
   al2o3 + coating: 324 registros
   silica + coating: 282 registros
   sio2 + coating: 279 registros
   graphene + coating: 256 registros
   tio2 + coatings: 239 registros
   al2o3 + coatings: 238 registros
   alumina + coating: 216 registros
   nanocomposite + coating: 213 registros
   zno + coating: 203 registros
   zirconia + coating: 182 registros
   epoxy + coating: 176 registros
   silver + coating: 174 registros
   sio2 + coatings: 166 registros
   zirconia + coatings: 164 registros
   alumina + coatings: 160 registros
   silica + coatings: 158 registros
   hydroxyapatite + coating: 149 registros
   nanocomposite + coatings: 143 registros
   tio2 + photocatalytic: 142 registros
🎯 IDENTIFICANDO REGISTROS DE ALTA RELEVÂNCIA
📊 Registros com alta relevância nano (≥10.0): 1,168
📊 Registros com alta relevância funcional (≥5.0): 6,529
🎯 Registros com AMBAS as relevâncias altas: 1,006

📈 DISTRIBUIÇÃO POR CATEGORIA:
   high_func

## 💾 Salvamento dos Resultados da Análise Regex

In [21]:
def save_regex_results(df: pd.DataFrame, stats: Dict, cooccurrences: Dict) -> Dict[str, str]:
    """Salva todos os resultados da análise regex"""
    
    print("💾 SALVANDO RESULTADOS DA ANÁLISE REGEX")
    print("=" * 40)
    
    # Criar diretório se não existir
    processed_dir = PATHS.get('processed', './processed/')
    os.makedirs(processed_dir, exist_ok=True)
    
    # Caminhos dos arquivos
    files_saved = {}
    
    # 1. Dataset com análise regex
    dataset_path = os.path.join(processed_dir, 'dataset_with_regex_analysis.csv')
    df.to_csv(dataset_path, index=False, encoding='utf-8-sig')
    files_saved['dataset'] = dataset_path
    print(f"✅ Dataset salvo: {dataset_path}")
    
    # 2. Registros de alta relevância
    high_relevance = df[df['relevance_category'].isin(['high_nano', 'high_functional', 'high_both'])]
    high_relevance_path = os.path.join(processed_dir, 'high_relevance_records.csv')
    high_relevance.to_csv(high_relevance_path, index=False, encoding='utf-8-sig')
    files_saved['high_relevance'] = high_relevance_path
    print(f"✅ Registros de alta relevância salvos: {high_relevance_path}")
    
    # 3. Estatísticas da análise
    stats_path = os.path.join(processed_dir, 'regex_analysis_statistics.json')
    with open(stats_path, 'w', encoding='utf-8') as f:
        json.dump(stats, f, ensure_ascii=False, indent=2, default=str)
    files_saved['statistics'] = stats_path
    print(f"✅ Estatísticas salvas: {stats_path}")
    
    # 4. Análise de co-ocorrências
    cooccurrence_path = os.path.join(processed_dir, 'cooccurrence_analysis.json')
    with open(cooccurrence_path, 'w', encoding='utf-8') as f:
        json.dump(cooccurrences, f, ensure_ascii=False, indent=2, default=str)
    files_saved['cooccurrences'] = cooccurrence_path
    print(f"✅ Co-ocorrências salvas: {cooccurrence_path}")
    
    # 5. Resumo da análise
    summary = {
        'timestamp': datetime.now().isoformat(),
        'notebook': '03_analise_regex.ipynb',
        'total_records': len(df),
        'high_relevance_records': len(high_relevance),
        'nanomaterials_found': len(stats['nanomateriais_frequencia']),
        'functional_properties_found': len(stats['propriedades_frequencia']),
        'synthesis_methods_found': len(stats['metodos_sintese_frequencia']),
        'files_generated': files_saved,
        'next_notebook': '04_analise_gemini.ipynb'
    }
    
    summary_path = os.path.join(processed_dir, 'regex_analysis_summary.json')
    with open(summary_path, 'w', encoding='utf-8') as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    files_saved['summary'] = summary_path
    print(f"✅ Resumo salvo: {summary_path}")
    
    return files_saved

# Salvar todos os resultados
saved_files = save_regex_results(df_with_relevance, regex_stats, cooccurrence_analysis)

print(f"\n🎉 ANÁLISE REGEX CONCLUÍDA COM SUCESSO!")
print(f"📊 Total de registros processados: {len(df_with_relevance):,}")
print(f"🎯 Registros de alta relevância: {len(df_with_relevance[df_with_relevance['relevance_category'] != 'low']):,}")
print(f"💾 Arquivos gerados: {len(saved_files)}")
print(f"\n📁 Arquivos salvos:")
for desc, path in saved_files.items():
    print(f"   {desc}: {path}")

💾 SALVANDO RESULTADOS DA ANÁLISE REGEX
✅ Dataset salvo: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_with_regex_analysis.csv
✅ Dataset salvo: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_with_regex_analysis.csv
✅ Registros de alta relevância salvos: /home/delon/Modelos/modeloCenanoInk/data/processed/high_relevance_records.csv
✅ Estatísticas salvas: /home/delon/Modelos/modeloCenanoInk/data/processed/regex_analysis_statistics.json
✅ Co-ocorrências salvas: /home/delon/Modelos/modeloCenanoInk/data/processed/cooccurrence_analysis.json
✅ Resumo salvo: /home/delon/Modelos/modeloCenanoInk/data/processed/regex_analysis_summary.json

🎉 ANÁLISE REGEX CONCLUÍDA COM SUCESSO!
📊 Total de registros processados: 9,046
🎯 Registros de alta relevância: 6,691
💾 Arquivos gerados: 5

📁 Arquivos salvos:
   dataset: /home/delon/Modelos/modeloCenanoInk/data/processed/dataset_with_regex_analysis.csv
   high_relevance: /home/delon/Modelos/modeloCenanoInk/data/processed/high_relevance_r

In [22]:
def gerar_relatorio_regex(df: pd.DataFrame) -> Dict[str, Any]:
    """
    Gera relatório detalhado da análise regex.
    
    Args:
        df: DataFrame com análise regex
    
    Returns:
        Dicionário com estatísticas
    """
    def convert_numpy_types(obj):
        """Converte tipos NumPy para tipos Python nativos para serialização JSON"""
        if hasattr(obj, 'item'):
            return obj.item()
        elif hasattr(obj, 'tolist'):
            return obj.tolist()
        elif isinstance(obj, dict):
            return {k: convert_numpy_types(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(v) for v in obj]
        else:
            return obj
    
    if df.empty:
        return {}
    
    # Usar as colunas corretas baseadas na análise atual
    relatorio = {
        'total_artigos_analisados': int(len(df)),
        'distribuicao_escopo': convert_numpy_types(df['relevance_category'].value_counts().to_dict() if 'relevance_category' in df.columns else {}),
        'estatisticas_nanomateriais': {
            'media_score': float(df['nano_relevance_score'].mean() if 'nano_relevance_score' in df.columns else 0),
            'mediana_score': float(df['nano_relevance_score'].median() if 'nano_relevance_score' in df.columns else 0),
            'max_score': float(df['nano_relevance_score'].max() if 'nano_relevance_score' in df.columns else 0),
            'artigos_com_nanomateriais': int((df['nano_relevance_score'] > 0).sum() if 'nano_relevance_score' in df.columns else 0)
        },
        'estatisticas_funcionais': {
            'media_score': float(df['functional_relevance_score'].mean() if 'functional_relevance_score' in df.columns else 0),
            'mediana_score': float(df['functional_relevance_score'].median() if 'functional_relevance_score' in df.columns else 0),
            'max_score': float(df['functional_relevance_score'].max() if 'functional_relevance_score' in df.columns else 0),
            'artigos_com_funcionais': int((df['functional_relevance_score'] > 0).sum() if 'functional_relevance_score' in df.columns else 0)
        },
        'top_nanomateriais': {},
        'top_propriedades_funcionais': {}
    }
    
    # Extrair nanomateriais mais frequentes
    if 'nanomateriais_encontrados' in df.columns:
        todos_nanomateriais = []
        for nanomateriais_json in df['nanomateriais_encontrados'].dropna():
            try:
                nanomateriais = json.loads(nanomateriais_json)
                todos_nanomateriais.extend(nanomateriais)
            except:
                pass
        
        if todos_nanomateriais:
            contador_nanomateriais = Counter(todos_nanomateriais)
            relatorio['top_nanomateriais'] = dict(contador_nanomateriais.most_common(10))
    
    # Extrair propriedades funcionais mais frequentes
    if 'propriedades_funcionais' in df.columns:
        todas_propriedades = []
        for propriedades_json in df['propriedades_funcionais'].dropna():
            try:
                propriedades = json.loads(propriedades_json)
                todas_propriedades.extend(propriedades)
            except:
                pass
        
        if todas_propriedades:
            contador_propriedades = Counter(todas_propriedades)
            relatorio['top_propriedades_funcionais'] = dict(contador_propriedades.most_common(10))
    
    return relatorio

# Gerar relatório usando os dados corretos
if not df_with_relevance.empty:
    relatorio_regex = gerar_relatorio_regex(df_with_relevance)
    
    # Criar diretório se não existir
    processed_dir = PATHS.get('processed', './processed/')
    os.makedirs(processed_dir, exist_ok=True)
    
    # Salvar relatório
    relatorio_path = os.path.join(processed_dir, 'relatorio_analise_regex.json')
    with open(relatorio_path, 'w', encoding='utf-8') as f:
        json.dump(relatorio_regex, f, ensure_ascii=False, indent=2)
    
    print("📊 Relatório da Análise Regex:")
    print(f"  Total de artigos analisados: {relatorio_regex['total_artigos_analisados']:,}")
    print("  Distribuição por categoria de relevância:")
    for categoria, count in relatorio_regex['distribuicao_escopo'].items():
        print(f"    {categoria}: {count:,} artigos")
    
    print("  Estatísticas de nanomateriais:")
    stats_nano = relatorio_regex['estatisticas_nanomateriais']
    print(f"    Artigos com nanomateriais: {stats_nano['artigos_com_nanomateriais']:,}")
    print(f"    Score médio: {stats_nano['media_score']:.2f}")
    
    print("  Top nanomateriais encontrados:")
    for nanomaterial, count in list(relatorio_regex['top_nanomateriais'].items())[:5]:
        print(f"    {nanomaterial}: {count:,}")
    
    print(f"💾 Relatório salvo em '{relatorio_path}'")
else:
    print("⚠️  Dados não disponíveis para relatório")
    relatorio_regex = {}

📊 Relatório da Análise Regex:
  Total de artigos analisados: 9,046
  Distribuição por categoria de relevância:
    high_functional: 5,523 artigos
    low: 2,355 artigos
    high_both: 1,006 artigos
    high_nano: 162 artigos
  Estatísticas de nanomateriais:
    Artigos com nanomateriais: 4,235
    Score médio: 3.94
  Top nanomateriais encontrados:
    tio2: 550
    al2o3: 446
    sio2: 383
    silica: 382
    graphene: 334
💾 Relatório salvo em '/home/delon/Modelos/modeloCenanoInk/data/processed/relatorio_analise_regex.json'


## 🔄 Preparação para Próxima Etapa (Análise Gemini)

# Verificar se o sistema está pronto para a próxima etapa
ready_for_gemini = True
ready_checks = []

# Verificar arquivos necessários
required_files = [
    'dataset_with_regex_analysis.csv',
    'high_relevance_records.csv', 
    'regex_analysis_statistics.json'
]

processed_dir = PATHS.get('processed', './processed/')
for filename in required_files:
    filepath = os.path.join(processed_dir, filename)
    if os.path.exists(filepath):
        ready_checks.append(f"✅ {filename}")
    else:
        ready_checks.append(f"❌ {filename}")
        ready_for_gemini = False

# Verificar configuração da API Gemini
if CONFIG.get('gemini', {}).get('api_key'):
    ready_checks.append("✅ API Gemini configurada")
else:
    ready_checks.append("❌ API Gemini não configurada")
    ready_for_gemini = False

# Atualizar configuração do sistema
config_data['regex_analysis'] = {
    'completed': True,
    'timestamp': datetime.now().isoformat(),
    'records_processed': len(df_with_relevance),
    'high_relevance_records': len(df_with_relevance[df_with_relevance['relevance_category'] != 'low']),
    'files_generated': list(saved_files.keys()),
    'ready_for_gemini': ready_for_gemini
}

# Salvar configuração atualizada
with open('config_sistema.json', 'w', encoding='utf-8') as f:
    json.dump(config_data, f, ensure_ascii=False, indent=2)

print("🔄 PREPARAÇÃO PARA PRÓXIMA ETAPA")
print("=" * 35)
print("\n📋 Verificações:")
for check in ready_checks:
    print(f"   {check}")

if ready_for_gemini:
    print("\n✅ SISTEMA PRONTO PARA ANÁLISE GEMINI!")
    print("🚀 Próximo notebook: 04_analise_gemini.ipynb")
else:
    print("\n⚠️ Sistema não está completamente pronto para análise Gemini")
    print("💡 Verifique as configurações e arquivos faltantes acima")

print(f"\n📊 RESUMO FINAL:")
print(f"   📁 Registros processados: {len(df_with_relevance):,}")
print(f"   🎯 Alta relevância nano: {len(df_with_relevance[df_with_relevance['relevance_category'].isin(['high_nano', 'high_both'])]):,}")
print(f"   🔧 Alta relevância funcional: {len(df_with_relevance[df_with_relevance['relevance_category'].isin(['high_functional', 'high_both'])]):,}")
print(f"   ⭐ Ambas relevâncias altas: {len(df_with_relevance[df_with_relevance['relevance_category'] == 'high_both']):,}")
print(f"   🧪 Nanomateriais únicos: {len(regex_stats['nanomateriais_frequencia'])}")
print(f"   ⚙️ Propriedades funcionais únicas: {len(regex_stats['propriedades_frequencia'])}")

print("\n🎉 NOTEBOOK 03 - ANÁLISE REGEX CONCLUÍDO!")

In [23]:
# Atualizar metadados para próxima etapa
metadados_regex = {
    'notebook': '03_analise_regex',
    'timestamp': pd.Timestamp.now().isoformat(),
    'dados_processados': os.path.join(PATHS.get('processed', './processed/'), 'dataset_with_regex_analysis.csv'),
    'relatorio': os.path.join(PATHS.get('processed', './processed/'), 'relatorio_analise_regex.json'),
    'total_artigos': len(df_with_relevance) if 'df_with_relevance' in locals() and not df_with_relevance.empty else 0,
    'colunas_adicionadas': [
        'nano_relevance_score',
        'functional_relevance_score',
        'total_matches',
        'nanomateriais_encontrados',
        'propriedades_funcionais',
        'metodos_sintese',
        'medidas_encontradas',
        'analise_completa',
        'relevance_category'
    ],
    'proxima_etapa': '04_analise_gemini.ipynb'
}

# Salvar metadados
metadados_path = os.path.join(PATHS.get('processed', './processed/'), 'metadados_analise_regex.json')
with open(metadados_path, 'w', encoding='utf-8') as f:
    json.dump(metadados_regex, f, ensure_ascii=False, indent=2)

print("✅ Notebook 03 - Análise Regex concluído")
print(f"📊 {metadados_regex['total_artigos']:,} artigos processados com análise regex")
print("🔄 Próxima etapa: 04_analise_gemini.ipynb")
print(f"💾 Metadados salvos em '{metadados_path}'")

# Exibir resumo final
if 'df_with_relevance' in locals() and not df_with_relevance.empty:
    print("\n🎯 RESUMO FINAL DA ANÁLISE:")
    print(f"   📁 Total de registros: {len(df_with_relevance):,}")
    if 'relevance_category' in df_with_relevance.columns:
        category_counts = df_with_relevance['relevance_category'].value_counts()
        for category, count in category_counts.items():
            print(f"   {category}: {count:,} registros")
    print(f"   🧪 Nanomateriais únicos: {len(regex_stats['nanomateriais_frequencia']) if 'regex_stats' in locals() else 0}")
    print(f"   ⚙️ Propriedades funcionais únicas: {len(regex_stats['propriedades_frequencia']) if 'regex_stats' in locals() else 0}")
else:
    print("\n⚠️ Dados não disponíveis para resumo final")

✅ Notebook 03 - Análise Regex concluído
📊 9,046 artigos processados com análise regex
🔄 Próxima etapa: 04_analise_gemini.ipynb
💾 Metadados salvos em '/home/delon/Modelos/modeloCenanoInk/data/processed/metadados_analise_regex.json'

🎯 RESUMO FINAL DA ANÁLISE:
   📁 Total de registros: 9,046
   high_functional: 5,523 registros
   low: 2,355 registros
   high_both: 1,006 registros
   high_nano: 162 registros
   🧪 Nanomateriais únicos: 92
   ⚙️ Propriedades funcionais únicas: 50
